In [14]:
import sys
import numpy as np
import pandas as pd
sys.path.append(r'C:\Users\kwon\github\recipe-1\models')
import db

final = db.oracle2df('FINAL_RECIPE')
 # 10행까지 테스트

In [133]:
test = final.iloc[:100].copy()

In [134]:
test.filter(like ='quantity').applymap(type).stack().unique()

for col in test.filter(like ='quantity').columns :
    test[col] = test[col].astype(float)

In [145]:
def abs_unit_change(unit):
        abs_df = pd.read_csv('absolute_unit.csv', delimiter= ' ')
        abs_dict = dict(zip(abs_df["단위"], abs_df["무게(g)"]))
        return abs_dict[unit]

def abs_unit_dict():
        abs_df = pd.read_csv('absolute_unit.csv', delimiter= ' ')
        return list(abs_df["단위"])

def relative_unit_dict():
        df = pd.read_csv(r'models\hj\change_unit\단위데이터.csv')
        rel_dict = {}
        for i in range(len(df)):
            if df.loc[i, "식재료"] not in rel_dict:
                rel_dict[df.loc[i, "식재료"]] = {df.loc[i, "단위"]: df.loc[i, "무게"]}
            else:
                if df.loc[i, "단위"] not in rel_dict[df.loc[i, "식재료"]]:
                    rel_dict[df.loc[i, "식재료"]].update({df.loc[i, "단위"]: df.loc[i, "무게"]})
        return rel_dict 

def relative_unit_change(ingre, unit):
    output = relative_unit[ingre][unit]
    if '~' in output :
        min, max = output.split('~')
        return (int(min)+int(max))/2
    else :
        return float(output)

abs_unit = abs_unit_dict()
relative_unit = relative_unit_dict() # 상대단위 딕셔너리 생성
# relative_unit_change('달걀', '개') : 55

In [152]:
test[test =='300g'].sum().sum()

0.0

In [147]:
음식은있지만단위가없음 = []
음식이아예없음 = {}
from tqdm import tqdm
def parse_unit(data):
    df = data.copy() 
    len_unit = df.filter(like = 'unit').shape[1] # 유닛의 개수
    
    exception_list = []
    for i in tqdm(range(1, len_unit)):
        ingredients, quantity, units_col  = df[f'ingredient{i}'], df[f'quantity{i}'], df[f'unit{i}']
        
        gram_columns = pd.DataFrame(columns=[f'gram{i}']) # 새로운 그램 칼럼 생성하기
        data = pd.concat([data, gram_columns], axis=1)

        for j, (ingred, quant,  unit) in tqdm(enumerate(zip(ingredients, quantity, units_col))):
            #1. 단위가 숫자인 경우
            if type(unit) in (float, int) : 
                data.loc[j, f'unit{j}'] = float(quant) * float(unit)

            #2. 단위가 절대단위인 경우 
            elif unit in abs_unit : #절대단위 
                data.loc[j, f'unit{j}'] = float(quant)* abs_unit_change(unit)
                #print(f"{ingredient, unit}이 {absolute_unit(unit)}으로 변환됨")
               
            #3. 단위가 상대단위인 경우
            elif ingred in relative_unit: #상대단위
                if unit in relative_unit[ingred]:
                    data.loc[j, f'unit{j}'] = float(quant) * relative_unit_change(ingred, unit)
                    #print(f"{ingredient, unit}이 {dic_test[ingredient][unit]}으로 변환됨")
                else :
                    print(f" {ingred, unit}")
                    음식은있지만단위가없음.append([ingred, unit])
            elif ingred == 'nan' : pass
            else : 
                if ingred in 음식이아예없음:
                    음식이아예없음[ingred].append(unit)
                else :
                    음식이아예없음[ingred] = [unit]
    return df

parse_unit(test)
음식이아예없음

14it [00:00, 2307.28it/s]:00<?, ?it/s]
  0%|          | 0/23 [00:00<?, ?it/s]


ValueError: invalid literal for int() with base 10: '300g'

In [119]:
test.shape

(10, 86)

TypeError: 'module' object is not callable